<a href="https://colab.research.google.com/github/mwb05/331_Final_Project/blob/main/Final_Project_331.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Ask for file names
adp_filename = input("What is the file name for the ADP report? ") + ".csv"
grow_filename = input("What is the file name for the GROW report? ") + ".csv"

# Load the data
adp_report = pd.read_csv(adp_filename, skiprows=1)
grow_report = pd.read_csv(grow_filename)

# Prepare and clean the names in ADP report
adp_report['Person Name'] = adp_report['Person Name'].str.replace(',', '').str.strip().str.upper()

# Sum hours in ADP report by person
adp_report['Total_Hours'] = adp_report['Regular'].fillna(0) + adp_report['Overtime'].fillna(0) + adp_report['Doubletime'].fillna(0)
adp_hours = adp_report.groupby('Person Name')['Total_Hours'].sum().reset_index()

# Prepare and clean the names in GROW report
grow_report['Full_Name'] = (grow_report['Last_Name'] + ' ' + grow_report['First_Name']).str.strip().str.upper()

# Sum hours in GROW report by person, including Work_Location_Id
grow_hours = grow_report.groupby(['Full_Name', 'Work_Location_Id'])['Hours_Worked'].sum().reset_index()

# Merge the datasets on the person's name and Work_Location_Id
merged_data = pd.merge(adp_hours, grow_hours, left_on='Person Name', right_on='Full_Name', how='inner')

# Calculate discrepancies
merged_data['Discrepancy'] = merged_data['Total_Hours'] - merged_data['Hours_Worked']

# Department and Manager info embedded directly
department_info = {
    303: 'TRAINING',
    5555: 'Gas Station',
    141803: 'Training Hawthorne',
    147003: 'Training Hollywood',
    157903: 'Training Westminster',
    159703: 'Training Westchester',
    174003: 'Training Los Alamitos',
    268803: 'Training Bixby Knolls',
    281603: 'Training Mid-Cities',
    355003: 'Training Bellflower',
    358003: 'Training Lynwood',
    372403: 'Training La Habra',
    374703: 'Training Bell',
    422603: 'Training_Western',
    555503: 'Training Gas Station',
    602003: 'Training Lakewood',
    611003: 'Training Los Angeles Manchester',
    706003: 'Training Los Angeles Sunset',
    31903: 'TRAINING_LITTLE ROCK_JFK BLVD',
    51503: 'TRAINING_LITTLE ROCK RODNEY PARHAM',
    372603: 'TRAINING LITTLE ROCK_BOWMAN RD.',
    381203: 'TRAINING BRYANT',
    408503: 'TRAINING CONWAY_OAK ST',
    408603: 'TRAINING CONWAY_DAVE WARD',
    408703: 'TRAINING MORRILTON',
    408803: 'TRAINING CONWAY_HOGAN LANE'
}

# Map the Work_Location_Id to department descriptions
merged_data['Dept Description'] = merged_data['Work_Location_Id'].map(department_info)

# Add a discrepancy check that considers department info
def check_discrepancy(row):
    dept_code = row['Work_Location_Id']  # Assuming this maps to department codes
    if dept_code in department_info:
        return 0  # No discrepancy if department info matches
    return row['Discrepancy']

merged_data['Adjusted Discrepancy'] = merged_data.apply(check_discrepancy, axis=1)

# Filter to show only rows with discrepancies
discrepancies = merged_data[merged_data['Adjusted Discrepancy'].abs() > 1]

if discrepancies.empty:
    print("There are no discrepancies today.")
else:
    print("Discrepancies found:")
    print(discrepancies)

    # Email setup
    sender = 'HRreports@ae-jl.com'
    password = 'kAj_yTjWDcL3v57'  # Change this to your actual password
    smtp_server = 'smtp.office365.com'
    port = 587

    server = smtplib.SMTP(smtp_server, port)
    server.starttls()
    server.login(sender, password)

    # Manager info embedded directly
     manager_info = {
    355: {'Manager': 'Fabian Hernandez', 'Manager Email': 'store00355@jiffylube.com', 'Area Manager': 'Tony Beas', 'Area Manager Email': 'tbeas@ae-jl.com'},
    602: {'Manager': 'Hamidu Hamid', 'Manager Email': 'store00602@jiffylube.com', 'Area Manager': 'Tony Beas', 'Area Manager Email': 'tbeas@ae-jl.com'},
    611: {'Manager': 'Fernando Moreno', 'Manager Email': 'store00611@jiffylube.com', 'Area Manager': 'Tony Beas', 'Area Manager Email': 'tbeas@ae-jl.com'},
    1418: {'Manager': 'Johnnie Raigoza', 'Manager Email': 'store01418@jiffylube.com', 'Area Manager': 'Tony Beas', 'Area Manager Email': 'tbeas@ae-jl.com'},
    1579: {'Manager': 'Michael Boj', 'Manager Email': 'store01579@jiffylube.com', 'Area Manager': 'Tony Beas', 'Area Manager Email': 'tbeas@ae-jl.com'},
    1597: {'Manager': 'Jose Navarro', 'Manager Email': 'store01597@jiffylube.com', 'Area Manager': 'Tony Beas', 'Area Manager Email': 'tbeas@ae-jl.com'},
    2816: {'Manager': 'Michael Damiron', 'Manager Email': 'store02186@jiffylube.com', 'Area Manager': 'Tony Beas', 'Area Manager Email': 'tbeas@ae-jl.com'},
    358: {'Manager': 'Vidal Zamora', 'Manager Email': 'store00358@jiffylube.com', 'Area Manager': 'Freddy Siordia', 'Area Manager Email': 'fsiordia@ae-jl.com'},
    706: {'Manager': 'Jonathan Villela', 'Manager Email': 'store00706@jiffylube.com', 'Area Manager': 'Freddy Siordia', 'Area Manager Email': 'fsiordia@ae-jl.com'},
    1470: {'Manager': 'Galen Carroll', 'Manager Email': 'store01470@jiffylube.com', 'Area Manager': 'Freddy Siordia', 'Area Manager Email': 'fsiordia@ae-jl.com'},
    1740: {'Manager': 'William Pettus', 'Manager Email': 'mwbiddle1@gmail.com', 'Area Manager': 'Freddy Siordia', 'Area Manager Email': 'mbiddle@zagmail.gonzaga.edu'},
    2688: {'Manager': 'Bryant Fenske', 'Manager Email': 'store02688@jiffylube.com', 'Area Manager': 'Freddy Siordia', 'Area Manager Email': 'fsiordia@ae-jl.com'},
    3724: {'Manager': 'Carlos Jimenez', 'Manager Email': 'store03724@jiffylube.com', 'Area Manager': 'Freddy Siordia', 'Area Manager Email': 'fsiordia@ae-jl.com'},
    3747: {'Manager': 'Brigido Cisneros', 'Manager Email': 'store03747@jiffylube.com', 'Area Manager': 'Freddy Siordia', 'Area Manager Email': 'fsiordia@ae-jl.com'},
    4226: {'Manager': 'Giovanni Peralta', 'Manager Email': 'store04226@jiffylube.com', 'Area Manager': 'Freddy Siordia', 'Area Manager Email': 'fsiordia@ae-jl.com'},
    319: {'Manager': 'Seqoua Richard', 'Manager Email': 'store00319@jiffylube.com', 'Area Manager': 'Demetrius Haywood', 'Area Manager Email': 'dhaywood@ae-jl.com'},
    4085: {'Manager': 'Roy Holtz', 'Manager Email': 'store04085@jiffylube.com', 'Area Manager': 'Demetrius Haywood', 'Area Manager Email': 'dhaywood@ae-jl.com'},
    515: {'Manager': 'Devon Johnson', 'Manager Email': 'store00515@jiffylube.com', 'Area Manager': 'Demetrius Haywood', 'Area Manager Email': 'dhaywood@ae-jl.com'},
    4086: {'Manager': 'Dustin McCoy', 'Manager Email': 'store04086@jiffylube.com', 'Area Manager': 'Demetrius Haywood', 'Area Manager Email': 'dhaywood@ae-jl.com'},
    3726: {'Manager': 'Demetrius Haywood', 'Manager Email': 'store03726@jiffylube.com', 'Area Manager': 'Demetrius Haywood', 'Area Manager Email': 'dhaywood@ae-jl.com'},
    4087: {'Manager': 'Migual Peralta', 'Manager Email': 'store04087@jiffylube.com', 'Area Manager': 'Demetrius Haywood', 'Area Manager Email': 'dhaywood@ae-jl.com'},
    3812: {'Manager': 'Jerald Poindexter', 'Manager Email': 'store03812@jiffylube.com', 'Area Manager': 'Demetrius Haywood', 'Area Manager Email': 'dhaywood@ae-jl.com'},
    4088: {'Manager': 'Rodney Torrance', 'Manager Email': 'store04088@jiffylube.com', 'Area Manager': 'Demetrius Haywood', 'Area Manager Email': 'dhaywood@ae-jl.com'}}

    for index, row in discrepancies.iterrows():
        store_number = row['Work_Location_Id']
        if store_number in manager_info:
            manager_details = manager_info[store_number]
            receiver = manager_details['Manager Email']
            cc = [manager_details['Area Manager Email']]
            message = MIMEMultipart()
            message['From'] = sender
            message['To'] = receiver
            message['CC'] = ', '.join(cc)
            message['Subject'] = f"Discrepancy Report for Store {store_number} on {row['Work_Date']}"

            body = (f"Dear {manager_details['Manager']},\n\n"
                    f"Please review the following data for {row['Person Name']} on {row['Work_Date']}:\n\n"
                    f"ADP Reported Hours: {row['Total_Hours']} (Regular: {row['Regular']}, Overtime: {row['Overtime']}, Doubletime: {row['Doubletime']})\n"
                    f"GROW Reported Hours: {row['Hours_Worked']}\n"
                    f"Discrepancy Amount: {row['Adjusted Discrepancy']}\n\n"
                    f"Please research this discrepancy and respond with explanation to Valerie Dahl (vdahl@officeamg.com) within 48 hours. \n\n"
                    f"Thank you, The Alamitos Group HR Team")
            message.attach(MIMEText(body, 'plain'))

            recipients = [receiver] + cc
            text = message.as_string()
            server.sendmail(sender, recipients, text)

    server.quit()
